# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.io
from sklearn import svm, pipeline, base, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import rbf_kernel
import functools
import os.path, zipfile
import matplotlib.pyplot as plt
import warnings
from scipy import signal
import ImportantModules
from libsvm import commonutil as cu
from libsvm import svm as lsvm
from libsvm import svmutil 

# ChanVar Class

In [2]:
class ChanVar(base.BaseEstimator, base.TransformerMixin):
    def fit( X, y):return X,y
    def transform(X):
        return np.var(X, axis=1) 

# Loading DataSet

In [3]:
# Loading DataSet

# Read DataSet BCI Competition III, DataSet IVa - Training
sub1_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub1\100Hz\data_set_IVa_aa.mat", struct_as_record=True)
sub2_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub2\100Hz\data_set_IVa_al.mat", struct_as_record=True)
sub3_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub3\100Hz\data_set_IVa_av.mat", struct_as_record=True)
sub4_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub4\100Hz\data_set_IVa_aw.mat", struct_as_record=True)
sub5_100hz_training = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\100 Hz\MATLAB\sub5\100Hz\data_set_IVa_ay.mat", struct_as_record=True)


# Read DataSet BCI Competition III, DataSet IVa - True Label
sub1_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_aa.mat", struct_as_record=True)
sub2_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_al.mat", struct_as_record=True)
sub3_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_av.mat", struct_as_record=True)
sub4_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_aw.mat", struct_as_record=True)
sub5_100hz_true_label = scipy.io.loadmat(r"C:\Users\Daksh kumar\BCI CodeBase\Relevant DataSet\Competition III\4A\True Labels\true_labels_ay.mat", struct_as_record=True)


 Exctracting Important Information as Global Variables

In [4]:
# SciPy.io.loadmat does not deal well with Matlab structures, resulting in lots of
# extra dimensions in the arrays. This makes the code a bit more cluttered


m = sub1_100hz_training



sample_rate = m['nfo']['fs'][0][0][0][0]
#EEG = m['cnt'].T
#nchannels_yt, nsamples_yt = EEG.shape

#channel_names = [s[0] for s in m['nfo']['clab'][0][0][0]]
event_onsets = m['mrk'][0][0][0]
event_codes = m['mrk'][0][0][1]
#labels = np.zeros((1, nsamples), int)
#labels[0, event_onsets] = event_codes

cl_lab = [s[0] for s in m['mrk']['className'][0][0][0]]
cl1 = cl_lab[0]
cl2 = cl_lab[1]
nclasses = len(cl_lab)
nevents = len(event_onsets)

In [5]:
all_regions = [1,2,3,4,5]
#all_regions.remove(5)
#all_regions
#for i in range(0,len(all_regions)):
#    print(i)
#    print(all_regions[i])

Integrating Function - RFE

In [6]:
def Integration_CSP(all_regions):
    
    # Stopping Condition
    if (len(all_regions)<1):
        return "No Regions Remaining"
    
    # Divide Data
    
    d = ImportantModules.load_mat(sub1_100hz_training,sub1_100hz_true_label,None)
    # Extract Trials
    
    trials = Extract_trials(d,all_regions)
    # Apply BPF to Extracted Trials
    
    trials_filt = BPF(trials,all_regions)
    # Apply CSP to Filtered Data
    
    CSP = Apply_CSP(trials_filt,all_regions,d)
    # Combine Data from cl1, cl2 to 1 Variable
    
    Combined_CSP = Combine_Data(CSP,all_regions)
    # Apply ChanVar to Training, Testing Data
    
    CSPVar = chanvar(Combined_CSP,all_regions)
    # Create X Data
    
    X_CSP,X_CSP_All = Create_X_Data(CSPVar,all_regions)
    # Create Y Label
    
    y = Create_Y_Label(trials_filt,all_regions)
    # Classifier and Splitting Data using Train Test Split
    
    print("Composite Kernel - Results")
    model_all,X_train_all,X_test_all,y_train,y_test = SVMClassifier_all(X_CSP_All,y,all_regions)
    SVMPredict_all(model_all,all_regions,X_test_all,y_test)
    # Classifier and Splitting Data using Train Test Split
    
    model,X_train,X_test,y_train,y_test = SVMClassifier(X_CSP,y,all_regions)
    # Predicting Accuracy and other metrics
    
    SVMPredict(model,all_regions,X_test,y_test)
    # Kernel Calculator
    
    Kernel = Kernel_Calculator(X_train,all_regions)
    # Quadratic Norm Calculator
    
    Q_sorted = Quadratic_Norm_Calculator(Kernel,model,all_regions)
    # RFE
    print("Regions_Remaining")
    print(all_regions)
    # Another Stopping Condition
    if (len(all_regions)==1):
        print("Only 1 Region Remaining " + str(all_regions[0]))
        return True
    all_regions = RFE(Q_sorted,all_regions)
    # Recursion
    Integration_CSP(all_regions)

In [7]:
def Extract_trials(d,all_regions):
    trials = dict()
    for i in range(0,len(all_regions)):
        trials['Region' + str(all_regions[i])] = ImportantModules.ExtractTrial(d['Region' + str(all_regions[i])]['X'],event_onsets,sample_rate,cl_lab,event_codes)
    return trials
    

In [8]:
def BPF(trials,all_regions):
    trials_filt = dict()
    for i in range(0,len(all_regions)):
        trials_filt['Region' + str(all_regions[i])] = {cl1: ImportantModules.bandpass(trials['Region' + str(all_regions[i])][cl1], 5, 20, sample_rate),
               cl2: ImportantModules.bandpass(trials['Region' + str(all_regions[i])][cl2], 5, 20, sample_rate)}
    return trials_filt

In [9]:
def Apply_CSP(trials_filt,all_regions,d):
    W = dict()
    for i in range(0,len(all_regions)):
        W[str(all_regions[i])] = ImportantModules.csp(trials_filt['Region' + str(all_regions[i])][cl1],trials_filt['Region' + str(all_regions[i])][cl2])
    CSP = dict()
    for i in range(0,len(all_regions)):
        CSP['Region' + str(all_regions[i])] = {cl1: ImportantModules.apply_mix(W[str(all_regions[i])], trials_filt['Region' + str(all_regions[i])][cl1],np.array(d['Region'+str(all_regions[i])]['X'])),
              cl2: ImportantModules.apply_mix(W[str(all_regions[i])], trials_filt['Region' + str(all_regions[i])][cl2],np.array(d['Region'+str(all_regions[i])]['X']))}
    return CSP

In [10]:
def Combine_Data(CSP,all_regions):
    Combined_CSP = dict()
    for i in range(0,len(all_regions)):
        Combined_CSP['Region' + str(all_regions[i])] = np.dstack((CSP['Region' + str(all_regions[i])][cl1],CSP['Region' + str(all_regions[i])][cl2]))
    return Combined_CSP

In [11]:
def chanvar(Combined_CSP,all_regions):
    CSPVar = dict()
    for i in range(0,len(all_regions)):
        CSPVar['Region' + str(all_regions[i])] = ChanVar.transform(Combined_CSP['Region' + str(all_regions[i])])
    return CSPVar

In [12]:
def Create_X_Data(CSPVar,all_regions):
    X_CSP = dict()
    for i in range(0,len(all_regions)):
        X_CSP['Region' + str(all_regions[i])] = CSPVar['Region' + str(all_regions[i])].T
    #temp = np.empty((X_CSP['Region' + str(all_regions[i])].shape[0],118) )
    #print(temp.shape)
    X_CSP_all = np.hstack((X_CSP['Region' + str(all_regions[i])]) for i in range(0,len(all_regions)))
    #X_CSP_all
    print(X_CSP_all.shape,X_CSP_all)
    return X_CSP,X_CSP_all

In [13]:
def Create_Y_Label(trials_filt,all_regions):
    y_1 = np.zeros(trials_filt['Region1'][cl1].shape[2])
    y_1.fill(1)
    y_2 = np.zeros(trials_filt['Region1'][cl2].shape[2])
    y_2.fill(-1)
    y = np.hstack((y_1,y_2))
    #train_region1_combT = np.transpose(train_region1_comb,(2,1,0))
    print(y.shape)
    return y

In [14]:
def SVMClassifier(X_CSP,y,all_regions):
    X_train = dict()
    X_test = dict()
    y_train = dict()
    y_test = dict()
    for i in range(0,len(all_regions)):
        X_train['Region' + str(all_regions[i])],X_test['Region' + str(all_regions[i])],y_train,y_test = train_test_split(X_CSP['Region' + str(all_regions[i])],y,test_size=0.33,random_state= 42)
    
    model = dict()
    for i in range(0,len(all_regions)):
        model['Region' + str(all_regions[i])] = svm.SVC(kernel='rbf',gamma=1,shrinking=False)
        model['Region' + str(all_regions[i])].fit(X_train['Region' + str(all_regions[i])],y_train)
    return model,X_train,X_test,y_train,y_test

In [15]:
def SVMClassifier_all(X_CSP_all,y,all_regions):
    
    X_train,X_test,y_train,y_test = train_test_split(X_CSP_all,y,test_size=0.33,random_state= 42)
    model = svm.SVC(kernel='rbf',gamma=1,shrinking=False)
    model.fit(X_train,y_train)
    return model,X_train,X_test,y_train,y_test

In [16]:
def SVMPredict_all(model,all_regions,X_test,y_test):
    y_pred = model.predict(X_test)
    #print('Region' + str(all_regions[i]))
    print (metrics.classification_report(y_test, y_pred))
    print(accuracy_score(y_test,y_pred))

In [17]:
def SVMPredict(model,all_regions,X_test,y_test):
    for i in range(0,len(all_regions)):
        y_pred = model['Region' + str(all_regions[i])].predict(X_test['Region' + str(all_regions[i])])
        print('Region' + str(all_regions[i]))
        print (metrics.classification_report(y_test, y_pred))
        print(accuracy_score(y_test,y_pred))

In [18]:
def Kernel_Calculator(X_train,all_regions):
    Kernel = dict()
    for i in range(0,len(all_regions)):
        Kernel['Region' + str(all_regions[i])] = rbf_kernel(X_train['Region' + str(all_regions[i])].T)
    return Kernel

In [19]:
def Quadratic_Norm_Calculator(Kernel,model,all_regions):
    alpha = dict()
    for i in range(0,len(all_regions)):
        alpha['Region' + str(all_regions[i])] = np.mean(model['Region' + str(all_regions[i])].support_vectors_,axis = 0)
        alpha['Region' + str(all_regions[i])].resize((1,Kernel['Region' + str(all_regions[i])].shape[1]))
        print(alpha['Region' + str(all_regions[i])].shape)
    Q = dict()
    for i in range(0,len(all_regions)):
        Q['Region' + str(all_regions[i])] = np.dot(alpha['Region' + str(all_regions[i])],(np.dot(Kernel['Region' + str(all_regions[i])],alpha['Region' + str(all_regions[i])].T)))
    
    Q_sorted = {k: v for k, v in sorted(Q.items(), key=lambda item: item[1],reverse=True)}
    return Q_sorted

In [20]:
def RFE(Q_sorted,all_regions):
    print(all_regions)
    x = Q_sorted.popitem()
    print("Removing")
    print(x[0][-1])
    all_regions.remove(int(x[0][-1]))
    return all_regions

In [21]:
# Driver Function

In [22]:
all_regions = [1,2,3,4,5]
print(Integration_CSP(all_regions))

KeyboardInterrupt: 

In [ ]:
# For SCSP 
def Integration_SCSP(all_regions,mu,k):
    
    # Stopping Condition
    if (len(all_regions)<1):
        return "No Regions Remaining"
    
    # Divide Data
    
    d = ImportantModules.load_mat(sub1_100hz_training,sub1_100hz_true_label,None)
    # Extract Trials
    
    trials = Extract_trials(d,all_regions)
    # Apply BPF to Extracted Trials
    
    trials_filt = BPF(trials,all_regions)
    # Apply SCSP to Filtered Data
    
    SCSP = Apply_SCSP(trials_filt,all_regions,d,mu,k)
    # Combine Data from cl1, cl2 to 1 Variable
    
    Combined_SCSP = Combine_Data(SCSP,all_regions)
    # Apply ChanVar to Training, Testing Data
    
    SCSPVar = chanvar(Combined_SCSP,all_regions)
    # Create X Data
    
    X_SCSP,X_SCSP_All = Create_X_Data(SCSPVar,all_regions)
    # Create Y Label
    
    y = Create_Y_Label(trials_filt,all_regions)
    # Classifier and Splitting Data using Train Test Split
    
    model,X_train,X_test,y_train,y_test = SVMClassifier(X_SCSP,y,all_regions)
    print("Composite Kernel - Results")
    model_all,X_train_all,X_test_all,y_train,y_test = SVMClassifier_all(X_SCSP_All,y,all_regions)
    SVMPredict_all(model_all,all_regions,X_test_all,y_test)
    # Predicting Accuracy and other metrics
    
    print("RegionWise Results")
    SVMPredict(model,all_regions,X_test,y_test)
    # Kernel Calculator
    
    Kernel = Kernel_Calculator(X_train,all_regions)
    # Quadratic Norm Calculator
    
    Q_sorted = Quadratic_Norm_Calculator(Kernel,model,all_regions)
    # RFE
    print("Regions_Remaining")
    print(all_regions)
    # Another Stopping Condition
    
    if (len(all_regions)==1):
        print("Only 1 Region Remaining " + str(all_regions[0]))
        return True
    all_regions = RFE(Q_sorted,all_regions)
    # Recursion
    Integration_SCSP(all_regions,mu,k)

In [ ]:
def Apply_SCSP(trials_filt,all_regions,d,mu,k):
    W = dict()
    for i in range(0,len(all_regions)):
        W[str(all_regions[i])] = ImportantModules.scsp(trials_filt['Region' + str(all_regions[i])][cl1],trials_filt['Region' + str(all_regions[i])][cl2],mu,k)
    SCSP = dict()
    for i in range(0,len(all_regions)):
        SCSP['Region' + str(all_regions[i])] = {cl1: ImportantModules.apply_mix(W[str(all_regions[i])], trials_filt['Region' + str(all_regions[i])][cl1],np.array(d['Region'+str(all_regions[i])]['X'])),
              cl2: ImportantModules.apply_mix(W[str(all_regions[i])], trials_filt['Region' + str(all_regions[i])][cl2],np.array(d['Region'+str(all_regions[i])]['X']))}
    return SCSP

In [ ]:
# Driver Function
all_regions = [1,2,3,4,5]
mu = 0.8
k = 15
print(Integration_SCSP(all_regions,mu,k))